# 使用dash 在pynq3.0.1环境实时数据显示  

## 应升级以下3个包：   
python -m pip install --upgrade typing-extensions    由4.3.0 升级到 4.12.2   
python -m pip install --upgrade dash                 由2.6.0 升级到 3.0.0   
python -m pip install --upgrade plotly               由5.9.0 升级到 5.24.1

## 参考以下链接   
https://stackoverflow.com/questions/63589249/plotly-dash-display-real-time-data-in-smooth-animation

In [1]:
import dash
from dash import html
from dash import dcc
import numpy as np

from dash.dependencies import Input, Output

# Example data (a circle).
resolution = 20
t = np.linspace(0, np.pi * 2, resolution)
x, y = np.cos(t), np.sin(t)
# Example app.
figure = dict(data=[{'x': [], 'y': []}], layout=dict(xaxis=dict(range=[-1, 1]), yaxis=dict(range=[-1, 1])))
app = dash.Dash(__name__, update_title=None)  # remove "Updating..." from title
app.layout = html.Div([dcc.Graph(id='graph', figure=figure), dcc.Interval(id="interval")])


@app.callback(Output('graph', 'extendData'), [Input('interval', 'n_intervals')])
def update_data(n_intervals):
    if n_intervals is not None:
        index = n_intervals % resolution
        # tuple is (dict of new data, target trace index, number of points to keep)
        return dict(x=[[x[index]]], y=[[y[index]]]), [0], 10


if __name__ == '__main__':
    # 10.168.2.134 远程 ip
    app.run(host='10.168.2.134', port=8051, debug=True)

In [2]:
import dash
from dash import html
from dash import dcc
import numpy as np

from dash.dependencies import Input, Output, State

# Example data (a circle).
resolution = 1000
t = np.linspace(0, np.pi * 2, resolution)
x, y = np.cos(t), np.sin(t)
# Example app.
figure = dict(data=[{'x': [], 'y': []}], layout=dict(xaxis=dict(range=[-1, 1]), yaxis=dict(range=[-1, 1])))
app = dash.Dash(__name__, update_title=None)  # remove "Updating..." from title
app.layout = html.Div([
    dcc.Graph(id='graph', figure=dict(figure)), dcc.Interval(id="interval", interval=25),
    dcc.Store(id='offset', data=0), dcc.Store(id='store', data=dict(x=x, y=y, resolution=resolution)),
])
app.clientside_callback(
    """
    function (n_intervals, data, offset) {
        offset = offset % data.x.length;
        const end = Math.min((offset + 10), data.x.length);
        return [[{x: [data.x.slice(offset, end)], y: [data.y.slice(offset, end)]}, [0], 500], end]
    }
    """,
    [Output('graph', 'extendData'), Output('offset', 'data')],
    [Input('interval', 'n_intervals')], [State('store', 'data'), State('offset', 'data')]
)

if __name__ == '__main__':
    app.run(host='10.168.2.134', port=8052, debug=True)